## **Import the libraries we’ll need during our model building phase.**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

## **Load data from your local drive**

Recall the pre-processing steps we discussed earlier. We’ll be using them here after loading the data.

In [ ]:
train = pd.read_csv('train.csv')

fromNext, we will read all the training images, store them in a list, and finally convert that list into a numpy array.

In [ ]:
import gc

train_image = []
img_paths = []
dim = 100

for i in range(train.shape[0]):
    img_paths.append('train\\train\\'+str(train['category'][i]).zfill(2)+'\\'+train['filename'][i])

for path in img_paths:
    img = image.load_img(path, grayscale=False, color_mode='grayscale', target_size=(dim,dim))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
    del img
    gc.collect()

X = np.array(train_image)

#dispose not-to-be-used memory
del train_image
del img_paths
gc.collect()

As it is a multi-class classification problem (42 classes), we will one-hot encode the target variable.

In [ ]:
y = train['category'].values
y = to_categorical(y)

#dispose not-to-be-used memory
del train
gc.collect()

## **Creating a validation set from the training data.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

#dispose not-to-be-used memory
del X
del y
gc.collect()

## **Define the model structure.**

We will create a simple architecture with 2 convolutional layers, one dense hidden layer and an output layer.

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(dim,dim,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(42, activation='softmax'))

Next, we will compile the model we’ve created.

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

## **Loading the model and other important objects for training**

We will load the necessary objects for model fitting/training to save the time loading if met with big data.

In [1]:
import pickle
from keras.models import load_model

main_dir = 'model_stor\\'

with open(main_dir+'X_train.dataset','rb') as file_stream_add:
    X_train = pickle.load(file_stream_add)
with open(main_dir+'y_train.dataset','rb') as file_stream_add2:
    y_train = pickle.load(file_stream_add2)
with open(main_dir+'X_test.dataset','rb') as file_stream_add3:
    X_test = pickle.load(file_stream_add3)
with open(main_dir+'y_test.dataset','rb') as file_stream_add4:
    y_test = pickle.load(file_stream_add4)
    
model = load_model(main_dir+'my_model.h5')

Using TensorFlow backend.


## **Training the model.**

In this step, we will train the model on the training set images and validate it using, you guessed it, the validation set.

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
2635/2635 [==============================] - 1439s 546ms/step - loss: 0.9182 - accuracy: 0.7111 - val_loss: 3.8387 - val_accuracy: 0.3074
Epoch 2/10
2635/2635 [==============================] - 1713s 650ms/step - loss: 0.8911 - accuracy: 0.7191 - val_loss: 3.4902 - val_accuracy: 0.3049
Epoch 3/10
2635/2635 [==============================] - 1665s 632ms/step - loss: 0.8678 - accuracy: 0.7279 - val_loss: 3.4826 - val_accuracy: 0.2956
Epoch 4/10
 539/2635 [=====>........................] - ETA: 19:55 - loss: 0.8063 - accuracy: 0.7443

## **Saving the model and other important objects for training**

We will save the necessary objects for model fitting/training to save the time loading if met with big data.

In [ ]:
import pickle

main_dir = 'model_stor\\'

with open(main_dir+'X_train.dataset','wb') as file_stream_add:
    pickle.dump(X_train, file_stream_add)
with open(main_dir+'y_train.dataset','wb') as file_stream_add2:
    pickle.dump(y_train, file_stream_add2)
with open(main_dir+'X_test.dataset','wb') as file_stream_add3:
    pickle.dump(X_test, file_stream_add3)
with open(main_dir+'y_test.dataset','wb') as file_stream_add4:
    pickle.dump(y_test, file_stream_add4)
model.save(main_dir+'my_model.h5')

## **Making predictions!**

We’ll initially follow the steps we performed when dealing with the training data. Load the test images and predict their classes using the model.predict_classes() function. We will read and store all the test images:

In [ ]:
import pandas as pd
import gc

dim = 100
test = pd.read_csv('test.csv')
test_image = []

for i in range(test.shape[0]):
    img = image.load_img('test\\test\\'+test['filename'][i], grayscale=False, color_mode='grayscale', target_size=(dim,dim))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
    del img
    gc.collect()
    
test = np.array(test_image)

In [ ]:
del test_image
gc.collect()

In [ ]:
# making predictions
prediction = model.predict_classes(test)

Then write the results of the prediction to a csv file.

In [ ]:
# creating results file
results = pd.read_csv('test.csv')
results['category'] = prediction
results.to_csv('predicted_test.csv', header=True, index=False)